# Retail Example


This example demonstrates how to use Graph-ND for a more controlled, precision-focused workflow. Unlike the quick-start components example, it avoids reliance on LLMs for data mapping and enables greater user control & flexibility for real-world use cases. Key highlights:
1. Provide a pre-defined graph schema instead of inferring via LLM
2. Use your own code to map tables to nodes and relationships rather than relying on an LLM-_still no need to write any Cypher!_
3. Define filtered target schemas for text extraction to improve data quality.
4. Add custom source metadata to comply with your workflows or data governance
5. Add custom graph retrieval tools to improve accuracy and tailor to your use case


To run this notebook:
1. **Set up Neo4j (Aura):**
    - Start a free Neo4j instance at [console.neo4j.io](https://console.neo4j.io/) and save the credentials file.

2. **Clone and navigate to the repo:**
    - `git clone https://github.com/zach-blumenfeld/graph-nd.git`
    - `cd graph-nd`

3. **Prepare your environment:**
    - Create a Python virtual environment and install dependencies:
`pip install -r requirements.txt`
    - Configure your `.env` file in `graph-nd/examples/retail/` with Neo4j credentials and your `OPENAI_API_KEY` as shown below.

4. **Run the notebook:**
    - Navigate to the appropriate folder: `graph-nd/examples/retail/`
    - Open and run `retail-example.ipynb`.



In [1]:
import os

parent_dir = os.getcwd()
data_dir = os.path.join(parent_dir, "data")
data_model_dir = os.path.join(parent_dir, "data-models")


## Setup

make sure to have a `.env` file with the below

In [18]:
from dotenv import load_dotenv
from getpass import getpass

load_dotenv('.env', override=True)

uri = os.getenv('NEO4J_URI')
username = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')

if not os.getenv('OPENAI_API_KEY'):
    api_key = getpass("Please enter your OpenAI API key: ")

## Drafting Graph Schemas
Creating graph schemas for production is an iterative process requiring reviews, version controls, and some trial and error.
Graph-ND is designed to support this process.
To get started you can create an initial graph schema from any JSON-like file. For example, you can start with other data modeling tools, such as the Neo4j Data Importer, and export the resulting schema to a file. GraphRAG can then map this to an initial graph schema, which experts can refine further as needed.


In [19]:
from graph_nd import GraphRAG
from langchain_openai import ChatOpenAI


#file names
json_file = os.path.join(data_model_dir, "neo4j-importer-draft.json")
graph_schema_v1 = os.path.join(data_model_dir, "graph-schema-v1.json")

# LLM
llm=ChatOpenAI(model="gpt-4o", temperature=0.0)

# draft v1 graph-schema from neo4j importer model
(GraphRAG(llm=llm).schema
 .from_json_like_file(json_file)
 .export(graph_schema_v1))

[Schema] Successfully Crafted schema


## Tracking & Loading Schemas
You can iterate, track, and re-load these graph schema files. Allowing you to have __precise, version controlled, expert crafted schemas__. Below is how you load a graph schema for use

In [20]:
from graph_nd import GraphRAG
from neo4j import GraphDatabase
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

db_client = GraphDatabase.driver(uri, auth=(username, password))
embedding_model = OpenAIEmbeddings(model='text-embedding-ada-002')
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
schema_file = os.path.join(data_model_dir, "graph-schema-finalized.json")

# instantiate graphrag
graphrag = GraphRAG(db_client, llm, embedding_model)

# load schema
graphrag.schema.load(schema_file)

[Schema] Schema successfully loaded from /Users/zachblumenfeld/demo/graphrag-nd/examples/retail/data-models/graph-schema-finalized.json


## Automated Table Mapping - It Isn't Wrong but It Misses Some Things
Below is how you would run Graph-ND using inferred table mappings. Feel free to run and experiment.  In this case it doesn't appear wrong, but it will likely miss some relationships and there are some properties in the graph schema which aren't present in the tables (require our own calculations).

It is likely that with more descriptions in the graph schema we can get the mappings to do better, but for this example we will show how you can incorporate your own mapping logic

In [23]:
# for csv in ["order-details.csv", "products.csv", "articles.csv", "customers.csv", "suppliers.csv"]:
#     graphrag.data.merge_csv(os.path.join(data_dir, csv))


[Data] Merging order-details.csv as TableTypeEnum.RELATIONSHIPS.
[Data] Merging products.csv as TableTypeEnum.SINGLE_NODE.
[Data] Merging articles.csv as TableTypeEnum.SINGLE_NODE.
[Data] Merging customers.csv as TableTypeEnum.SINGLE_NODE.
[Data] Merging suppliers.csv as TableTypeEnum.SINGLE_NODE.


## Map Tabular Data with Our Own Logic
We can map data with our own custom logic for precision before merging nodes and relationship records directly with `data.merge_nodes` and `data.merge_relationships` methods
We will provide a custom metadata field "ingest_id" which will persist to `__Source__` nodes in the graph.

In [24]:
#We will provide a custom metadata field "ingest_id" which will persist to __Source__ nodes in the graph.
ingest_id = "my-tabular-data-ingest"

In [25]:
# Lets clear all data out of the graph for reference
graphrag.data.nuke()

### Merge Nodes

`graphrag.data.merge_nodes` will take node records and perform an __idempotent__ MERGE into the graph.

Technically source metadata ( `__Source__` node and `__source_id` lists properties) aren't idempotent  - more similar to "append" pattern so you can track activity, but the node records themselves are merged idempotently.

Parameters:
- label str: The label of the node type to merge (e.g., "Person", "Movie").
             The label should match a defined node in the graph schema.
- records List[Dict]: A list of dictionaries representing the data for each node to be merged.
            Each record MUST include the `id` field as defined in the node schema, along with
            any other optional properties expected by the schema.
- source_metadata Union[bool, Dict[str, Any]], optional : Metadata for the source being merged.
    - If set to `True`, default source metadata is prepared and added to a __Source__ node in the graph.
    A __source_id property is added and/or appended to each node which maps to the id property of __Source__ node
    - If `False`, no source metadata is added to the graph.
    - If a custom dictionary is provided, source metadata is added as in the case of `True` and the dictionary properties override the default ones.
    Default is True.

Example:
```
    label = "Person"
    records = [
        {"person_id": 1, "name": "Alice", "age": 30},
        {"person_id": 2, "name": "Bob", "age": 25}
    ]
```


In [26]:
import pandas as pd

order_df = pd.read_csv(os.path.join(data_dir, "order-details.csv"))
order_records = order_df[['orderId', 'tDat']].drop_duplicates().rename(columns={'tDat':'date'}).to_dict(orient="records")

graphrag.data.merge_nodes("Order", order_records, source_metadata={"ingest_id": ingest_id})

### Merge Relationships

`graphrag.data.merge_relationships` Just will take relationship records and perform an __idempotent__ MERGE into the graph just like the node merge

Source metadata will be added in the same matter as well.

Parameters:
- rel_type str: The type of the relationship (e.g., "ACTED_IN", "FRIENDS_WITH").
                The type should match a defined relationship in the graph schema.
- start_node_label str: The label of the starting node in the relationship (e.g., "Person").
                        This label should match a defined node schema.
- end_node_label str: The label of the ending node in the relationship (e.g., "Movie").
                      This label should match a defined node schema.
- records Dict: A dictionary (or list of dictionaries) representing the data for each relationship to be merged.
- source_metadata Union[bool, Dict[str, Any]], optional: same as for nodes

Required Fields in `records`:
    - `start_node_id`: The unique identifier of the starting node.
    - `end_node_id`: The unique identifier of the ending node.

Example:

    rel_type = "ACTED_IN"
    start_node_label = "Person"
    end_node_label = "Movie"
    records = [
        {"start_node_id": 1, "end_node_id": "M101", "role": "Protagonist"},
        {"start_node_id": 2, "end_node_id": "M102", "role": "Hacker"}
    ]

In [27]:
ordered_records = order_df[['customerId', 'orderId']].drop_duplicates().rename(columns={'customerId':'start_node_id', 'orderId':'end_node_id'}).to_dict(orient="records")

graphrag.data.merge_relationships(rel_type='ORDERED',
                                  start_node_label='Customer',
                                  end_node_label='Order',
                                  records=ordered_records,
                                  source_metadata={"ingest_id": ingest_id})

### Merging Parallel Relationships
Parallel Relationships are when more than one relationship of the same type can exist between the same two start and end nodes.  The above example doesn't account for them.  For these situations you must add an "id" property field to the relationship schema.  We specified `txId` as the relationship id for the CONTAINS relationship - in this example, an Order can CONTAIN multiple instances of the same Article.

The txId is then required for the merge.  not providing it will result in an error.

In [28]:
contains_records = (order_df[['orderId', 'articleId', 'txId', 'price']]).rename(columns={'orderId':'start_node_id', 'articleId':'end_node_id'}).to_dict(orient="records")
graphrag.data.merge_relationships(rel_type='CONTAINS',
                                  start_node_label='Order',
                                  end_node_label='Article',
                                  records=contains_records,
                                  source_metadata={"ingest_id": ingest_id})

### Text Embedding
Graph-ND will automatically create text embeddings and vector indexes based of searchField properties in the graph schema.  This also enables the automatic creation of FullText fields though we don't show an example here.

In the Below the Product node embeds the "text" field per the GraphSchema.  This merge will take a bit longer due to that embedding processed

In [29]:
product_df = pd.read_csv(os.path.join(data_dir, "products.csv"))
product_df['text'] = ("##Product \n"
"Name: " + product_df['prodName'].fillna('') + "\n"
"Type: " + product_df['productTypeName'].fillna('') + "\n"
"Category: " + product_df['productGroupName'].fillna('') + "\n"
"Description: " + product_df['detailDesc'].fillna('')
)
product_df['url']= "https://xyzbrands/product/" + product_df['productCode'].astype(str)
product_df.rename(columns={"prodName": "name", "detailDesc":"description"}, inplace=True)


prod_records = product_df[[ "productCode", "name", "description", "url", "text"]].to_dict(orient="records")

print("This will take a minute or so because it is embedding the 'text' field.")
graphrag.data.merge_nodes("Product", prod_records, source_metadata={"ingest_id": ingest_id})

This will take a minute or so because it is embedding the 'text' field.


### Merging the Other Nodes and Relationships
The below demonstrates merging the rest of the structured data.  Note that no Cypher is required and it is order independent.  This allows you to focus on just creating the Node and relationship tables you want.  Graph-ND takes care of constraints, indexing, text embedding, etc.

In [30]:
prod_cat_records = (product_df[['productCode', 'productGroupName']]
                    .rename(columns={'productCode':'start_node_id', 'productGroupName':'end_node_id'})
                    .to_dict(orient="records"))

graphrag.data.merge_relationships(rel_type='PART_OF',
                                  start_node_label='Product',
                                  end_node_label='ProductCategory',
                                  records=prod_cat_records,
                                  source_metadata={"ingest_id": ingest_id})

In [31]:
prod_type_records = (product_df[['productCode', 'productTypeName']]
                    .rename(columns={'productCode':'start_node_id', 'productTypeName':'end_node_id'})
                    .to_dict(orient="records"))

graphrag.data.merge_relationships(rel_type='PART_OF',
                                  start_node_label='Product',
                                  end_node_label='ProductType',
                                  records=prod_type_records,
                                  source_metadata={"ingest_id": ingest_id})

In [32]:
customer_df = pd.read_csv(os.path.join(data_dir, "customers.csv"))
records = customer_df[[ "customerId", "postalCode", "age", "fashionNewsFrequency", "clubMemberStatus"]].to_dict(orient="records")

graphrag.data.merge_nodes("Customer", records, source_metadata={"ingest_id": ingest_id})

In [33]:
article_df = pd.read_csv(os.path.join(data_dir, "articles.csv"))

article_records = article_df[["articleId", "colourGroupCode", "colourGroupName", "graphicalAppearanceName", "graphicalAppearanceNo"]].to_dict(orient="records")

graphrag.data.merge_nodes("Article", article_records, source_metadata={"ingest_id": ingest_id})


In [34]:
variant_records = (article_df[['articleId', 'productCode']]
                    .rename(columns={'articleId':'start_node_id', 'productCode':'end_node_id'})
                    .to_dict(orient="records"))

graphrag.data.merge_relationships(rel_type='VARIANT_OF',
                                  start_node_label='Article',
                                  end_node_label='Product',
                                  records=variant_records,
                                  source_metadata={"ingest_id": ingest_id})

In [35]:
supplied_by_records = (article_df[['articleId', 'supplierId']]
                    .rename(columns={'articleId':'start_node_id', 'supplierId':'end_node_id'})
                    .to_dict(orient="records"))
graphrag.data.merge_relationships(rel_type='SUPPLIED_BY',
                                  start_node_label='Article',
                                  end_node_label='Supplier',
                                  records=supplied_by_records,
                                  source_metadata={"ingest_id": ingest_id})

In [36]:
supplier_df = pd.read_csv(os.path.join(data_dir, "suppliers.csv"))
supplier_records = supplier_df.rename(columns={"supplierName": "name", "supplierAddress": "address"}).to_dict(orient="records")
graphrag.data.merge_nodes("Supplier", supplier_records, source_metadata={"ingest_id": ingest_id})

## Text Extraction From PDF
We can Add schema subsets here for precision. The SubSchema has the following input options
- nodes: Union[str, List[str]], optional
    A node or list of node labels to include in the subset. If provided, the node schemas
    corresponding to these nodes will be retrieved.
- patterns: Union[Tuple[str, str, str], List[Tuple[str, str, str]]], optional
    A pattern or list of patterns defining relationships to filter by. Each pattern is a
    tuple containing:
    - Start node label (str)
    - Relationship type (str)
    - End node label (str)
    The relevant node schemas and relationship schemas will be included in the subset.
- relationships: Union[str, List[str]], optional
    A relationship type or list of relationship types to include in the subset.
    All query patterns for the relationship type (and their start and end nodes) will be included in the subset.
- description: str, optional
    A custom description for the subsetted graph schema. Can be used to pass customized prompts/context for extraction.
    If not provided, a default description may be generated based on the existing schema and provided subset criteria.

In [37]:
from graph_nd import SubSchema

# less than 1% of the time the llm will miss nodes/rels. This is random so repeating twice near guarantees we get everything
for i in range(2):
    graphrag.data.merge_pdf(os.path.join(data_dir, 'credit-notes.pdf'),
                            nodes_only=False,
                            sub_schema=SubSchema(
                                patterns=[('CreditNote','REFUND_FOR_ORDER', 'Order'), ('CreditNote',"REFUND_OF_ARTICLE", 'Article')]
                            ))

[Data] Merging data from document: /Users/zachblumenfeld/demo/graphrag-nd/examples/retail/data/credit-notes.pdf


Extracting entities from text: 100%|██████████| 31/31 [00:46<00:00,  1.49s/it]


Consolidating results...


Merging Relationships by Type & Pattern: 100%|██████████| 2/2 [00:01<00:00,  1.89rel/s]


[Data] Merging data from document: /Users/zachblumenfeld/demo/graphrag-nd/examples/retail/data/credit-notes.pdf


Extracting entities from text: 100%|██████████| 31/31 [00:47<00:00,  1.54s/it]


Consolidating results...


Merging Relationships by Type & Pattern: 100%|██████████| 2/2 [00:01<00:00,  1.85rel/s]


## Test an Agent

In [38]:
graphrag.agent("Which suppliers where responsible for the most refunds")

================================ Human Message =================================

Which suppliers where responsible for the most refunds
================================== Ai Message ==================================
Tool Calls:
  aggregate (call_It76SpJFvtaasfLCeuZbKWlc)
 Call ID: call_It76SpJFvtaasfLCeuZbKWlc
  Args:
    agg_instructions: Aggregate the number of refunds for each supplier by counting the REFUND_OF_ARTICLE relationships for each supplier and return the suppliers with the highest counts.
Running Query:
MATCH (cn:CreditNote)-[:REFUND_OF_ARTICLE]->(a:Article)-[:SUPPLIED_BY]->(s:Supplier)
RETURN s.name AS supplierName, COUNT(cn) AS refundCount
ORDER BY refundCount DESC
================================= Tool Message =================================
Name: aggregate

[
    {
        "supplierName": "1616 - Textile & Apparel Manufacturing",
        "refundCount": 45
    },
    {
        "supplierName": "1779 - Denim Textiles",
        "refundCount": 42
    },
    {
        "

## Adding Custom Tools For Retrieval
For our use case there may be some specific query templates and retrieval methodologies. We casn easily add these to our langgraph agent

In [39]:
from typing import List, Dict


def get_product_recommendations(product_codes_or_article_ids: List[int]) -> List[Dict]:
    """
    Retrieve product recommendations given a list of product codes or articles ids.
    Please re-order or filter further based on additional context from user.
    """
    res = db_client.execute_query("""
    //recommend from product codes
    MATCH (customer:Customer)-[:ORDERED]->()-[:CONTAINS]->()-[:VARIANT_OF]->
    (interestedInProducts:Product)<-[:VARIANT_OF]-(interestedInArticles:Article)<-[:CONTAINS]-()<-[:ORDERED]
    -(:Customer)-[:ORDERED]->()-[:CONTAINS]->(recArticle:Article)-[:VARIANT_OF]->(product:Product)
    WHERE (interestedInArticles.articleId IN $itemIds)
        OR (interestedInProducts.productCode IN $itemIds)
    WITH count(recArticle) AS recommendationScore, product
    RETURN product.productCode AS productCode,
        product.text AS text,
        product.url AS url
    ORDER BY recommendationScore DESC LIMIT 20
    """, itemIds=product_codes_or_article_ids, result_transformer_ = lambda r: r.data())
    return res


def get_product_order_supplier_info(product_codes: List[int]) -> List[Dict]:
    """
    Given a list of product codes, gets statistics for total orders and refunds as well as by supplier for each product.
    """
    res = db_client.execute_query("""
    MATCH(p:Product)<-[:VARIANT_OF]-(a:Article)-[:SUPPLIED_BY]->(s)
    WHERE p.productCode IN $productCodes
    WITH *,
      COUNT {MATCH (:Order)-[:CONTAINS]->(a)} AS numberOfOrders,
      COUNT {MATCH (:CreditNote)-[:REFUND_OF_ARTICLE]-(a)} AS numberOfRefunds
    RETURN p.productCode AS productCode,
      sum(numberOfOrders) AS totalOrders,
      sum(numberOfRefunds) AS totalReturns,
      collect({supplierId:s.supplierId, name:s.name, numberOfOrders:numberOfOrders, numberOfRefunds:numberOfRefunds}) AS supplierInfos
    """, productCodes=product_codes, result_transformer_ = lambda r: r.data())
    return res

def get_supplier_order_product_info(supplier_ids: List[int]) -> List[Dict]:
    """
    Given a list of supplier ids, gets statistics for the total orders and refunds as well by product delivered for each supplier.
    """
    res = db_client.execute_query("""
    MATCH(p:Product)<-[:VARIANT_OF]-(:Article)-[:SUPPLIED_BY]->(s)
    WHERE s.supplierId IN $supplierIds
    WITH DISTINCT p, s,
      COUNT {MATCH (:Order)-[:CONTAINS]->()-[:VARIANT_OF]->(p)} AS numberOfOrders,
      COUNT {MATCH (:CreditNote)-[:REFUND_OF_ARTICLE]-()-[:VARIANT_OF]->(p)} AS numberOfRefunds
    RETURN s.supplierId AS supplierId,
      sum(numberOfOrders) AS totalOrders,
      sum(numberOfRefunds) AS totalReturns,
      collect({productCode:p.productCode, name:s.name, numberOfOrders:numberOfOrders, numberOfRefunds:numberOfRefunds}) AS supplierInfos
    """, supplierIds=supplier_ids, result_transformer_ = lambda r: r.data())
    return res

In [42]:
agent = graphrag.create_react_agent(tools=[get_product_recommendations,
                                           get_product_order_supplier_info,
                                           get_supplier_order_product_info])

Now let's create a helper function to test out with multi-hop questions

In [43]:
from langchain_core.messages import HumanMessage

# use just like any other langgraph agent...we are going to make a wrapper function for convenience
config = {"configurable": {"thread_id": "thread-1"}}

def agent_stream(question, history=None):
    if history is None:
        history = list()
    for step in agent.stream(
        {"messages": history + [HumanMessage(content=question)]},
        stream_mode="values", config=config
    ):
        history.append(step["messages"][-1])
        step["messages"][-1].pretty_print()
    return history


In [44]:
history = agent_stream("What are some good sweaters for spring? Nothing too warm please!")

================================ Human Message =================================

What are some good sweaters for spring? Nothing too warm please!
================================== Ai Message ==================================
Tool Calls:
  node_search (call_CXueAweTUBTSwD70zSqJmDjo)
 Call ID: call_CXueAweTUBTSwD70zSqJmDjo
  Args:
    search_config: {'search_type': 'SEMANTIC', 'node_label': 'Product', 'search_prop': 'text'}
    search_query: spring sweaters
================================= Tool Message =================================
Name: node_search

[
    {
        "productCode": 921906,
        "text": "##Product \nName: Spring\nType: Dress\nCategory: Garment Full body\nDescription: Calf-length dress in an airy viscose weave with a collar, concealed buttons at the top and long raglan sleeves with buttoned cuffs. Relaxed fit with a gathered seam at the hips and above the hem. Unlined.",
        "description": "Calf-length dress in an airy viscose weave with a collar, concealed b

In [45]:
history = agent_stream("What else can you recommend to go with that?", history)

================================ Human Message =================================

What else can you recommend to go with that?
================================== Ai Message ==================================
Tool Calls:
  get_product_recommendations (call_LxVWTkiy6IOTCeKwHR3jLMpc)
 Call ID: call_LxVWTkiy6IOTCeKwHR3jLMpc
  Args:
    product_codes_or_article_ids: [793385, 574112, 796033, 620081, 671949]
================================= Tool Message =================================
Name: get_product_recommendations

[{"productCode": 744180, "text": "##Product \nName: Martini\nType: Dress\nCategory: Garment Full body\nDescription: Short wrap dress in woven fabric with short, wide sleeves and a V-neck with a concealed hook-and-eye fastener at the neckline. Seam and detachable tie belt at the waist and a gently flared skirt with a flounced trim. Unlined.", "url": "https://xyzbrands/product/744180"}, {"productCode": 817472, "text": "##Product \nName: Kelso\nType: Shorts\nCategory: Garment L

In [46]:
history2 = agent_stream("Which suppliers have the highest number of returns (i.,e, credit notes)?")

================================ Human Message =================================

Which suppliers have the highest number of returns (i.,e, credit notes)?
================================== Ai Message ==================================
Tool Calls:
  aggregate (call_H494cYQY3z1rEwzqUETKQCBn)
 Call ID: call_H494cYQY3z1rEwzqUETKQCBn
  Args:
    agg_instructions: Aggregate the number of credit notes (returns) for each supplier and return the suppliers with the highest number of returns.
Running Query:
MATCH (cn:CreditNote)-[:REFUND_OF_ARTICLE]->(a:Article)-[:SUPPLIED_BY]->(s:Supplier)
WITH s, count(cn) AS numberOfReturns
ORDER BY numberOfReturns DESC
RETURN s.name, numberOfReturns
================================= Tool Message =================================
Name: aggregate

[
    {
        "s.name": "1616 - Textile & Apparel Manufacturing",
        "numberOfReturns": 45
    },
    {
        "s.name": "1779 - Denim Textiles",
        "numberOfReturns": 42
    },
    {
        "s.name": "

In [47]:
history3 = agent_stream("What are the top 3 most returned products for supplier 1616? Get those product codes and find other suppliers who have less returns for each product I can use instead.")

================================ Human Message =================================

What are the top 3 most returned products for supplier 1616? Get those product codes and find other suppliers who have less returns for each product I can use instead.
================================== Ai Message ==================================
Tool Calls:
  aggregate (call_lrQg0Q7GjUHWWiApLnGgwjml)
 Call ID: call_lrQg0Q7GjUHWWiApLnGgwjml
  Args:
    agg_instructions: Find the top 3 most returned products for supplier 1616 by counting the number of refunds associated with each product supplied by this supplier. Return the product codes of these top 3 products.
Running Query:
MATCH (s:Supplier {supplierId: 1616})<-[:SUPPLIED_BY]-(a:Article)<-[:REFUND_OF_ARTICLE]-(c:CreditNote)
MATCH (a)-[:VARIANT_OF]->(p:Product)
RETURN p.productCode AS productCode, COUNT(c) AS refundCount
ORDER BY refundCount DESC
LIMIT 3
================================= Tool Message =================================
Name: aggregate


## MCP Integration
Of course, we can also use MCP to connect tools

In [ ]:
#TODO - This is super easy see https://github.com/langchain-ai/langchain-mcp-adapters?tab=readme-ov-file#client-1